In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA

In [5]:
with open('../docs/examples/state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

In [6]:
qa = VectorDBQA.from_llm(llm=OpenAI(), vectorstore=docsearch)

In [8]:
examples = [
    {
        "query": "What did the president say about Ketanji Brown Jackson",
        "answer": "He praised her legal ability and said he nominated her for the supreme court."
    },
    {
        "query": "What did the president say about Michael Jackson",
        "answer": "Nothing"
    }
]

In [10]:
predictions = qa.apply(examples)

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [14]:
template = """You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score it as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Please remember to grade them based on being factually accurate. Begin!

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:"""
prompt = PromptTemplate(input_variables=['query', 'result', "answer"], template=template)

In [15]:
llm = OpenAI(temperature=0)
grader = LLMChain(llm=llm, prompt=prompt)
grade_outputs = grader.apply(predictions)

In [17]:
predictions

[{'query': 'What did the president say about Ketanji Brown Jackson',
  'answer': 'He praised her legal ability and said he nominated her for the supreme court.',
  'result': " The president said that Ketanji Brown Jackson is one of the nation's top legal minds and is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."},
 {'query': 'What did the president say about Michael Jackson',
  'answer': 'Nothing',
  'result': ' The president did not mention Michael Jackson in this context.'}]

In [16]:
grade_outputs

[{'text': ' CORRECT'}, {'text': ' CORRECT'}]